## Import libraries

In [44]:
import os, sys

# add parent directory to path
sys.path.append("..")

In [45]:
import numpy as np
import cv2

import pandas as pd

from pedestrian_orientation import pose_estimator

import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

## Create Dataset

In [46]:
# get labels
labels_path = "../images/3DPes/3dpes_train.al"

# parse html file
import xml.etree.ElementTree as ET

tree = ET.parse(labels_path)
root = tree.getroot()

# get all annotations
annotations = root.findall("annotation")

# get all image names
image_paths = [annotation.find("image").find("name").text.strip() for annotation in annotations]

# get all labels
labels = [int(annotation.find("annorect").find("silhouette").find("id").text.strip()) for annotation in annotations]



In [47]:
dir_path = "../images/"

df_final = pd.DataFrame()

cnt = 0

for file_path, label in zip(image_paths, labels):
    file_path = os.path.join(dir_path, file_path)
    # extract filename from file_path
    filename = os.path.basename(file_path)

    # if it is folder continue
    if os.path.isdir(file_path):
        continue
    img = cv2.imread(file_path)
    if img is not None:
        landmarks = pose_estimator.get_pose_landmarks(img)

        if landmarks is None:
            # print("No landmarks found in image: ", filename)
            continue

        # add landmarks to dataframe
        df = pd.DataFrame(columns=["x", "y", "z", "visibility", "name"])
        for i, landmark in enumerate(landmarks.landmark):
            df.loc[i] = [
                landmark.x,
                landmark.y,
                landmark.z,
                landmark.visibility,
                mp_pose.PoseLandmark(i).name,
            ]

        # transform dataframe to long format
        df = df.melt(id_vars=["name"], var_name="axis", value_name="value")

        df["name"] = df["name"] + "_" + df["axis"]
        df = df.drop(columns=["axis"])

        # create pandas series from dataframe
        ds = df.set_index("name")["value"]

        # add cnt to series
        ds["file_count"] = cnt
        cnt += 1

        

        # add filename to series
        ds["filename"] = filename

        # add image height and width to series
        ds["img_height"] = img.shape[0]
        ds["img_width"] = img.shape[1]

        # add label to series
        ds["label"] = label

        # # add series as row to final dataframe using append
        df_final = df_final.append(ds, ignore_index=True)


        # add series as row to final dataframe using concat
        # df_final = pd.concat([df_final, pd.DataFrame(ds)], axis=1)

        # break        

C:\Users\Codrin\AppData\Local\Temp\ipykernel_17844\3984020696.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(ds, ignore_index=True)
C:\Users\Codrin\AppData\Local\Temp\ipykernel_17844\3984020696.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(ds, ignore_index=True)
C:\Users\Codrin\AppData\Local\Temp\ipykernel_17844\3984020696.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(ds, ignore_index=True)
C:\Users\Codrin\AppData\Local\Temp\ipykernel_17844\3984020696.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(ds, ign

In [48]:
df_final

name,NOSE_x,LEFT_EYE_INNER_x,LEFT_EYE_x,LEFT_EYE_OUTER_x,RIGHT_EYE_INNER_x,RIGHT_EYE_x,RIGHT_EYE_OUTER_x,LEFT_EAR_x,RIGHT_EAR_x,MOUTH_LEFT_x,...,RIGHT_ANKLE_visibility,LEFT_HEEL_visibility,RIGHT_HEEL_visibility,LEFT_FOOT_INDEX_visibility,RIGHT_FOOT_INDEX_visibility,file_count,filename,img_height,img_width,label
0,0.784383,0.795862,0.804042,0.811186,0.771800,0.762776,0.754197,0.821808,0.742224,0.797042,...,0.682844,0.267723,0.449096,0.545455,0.660193,0.0,102_23_FRAME_12_RGB.bmp,189,135,7
1,0.514450,0.525540,0.538651,0.551545,0.495063,0.485213,0.474640,0.587439,0.480625,0.543584,...,0.900378,0.271207,0.467985,0.515640,0.809789,1.0,102_23_FRAME_54_RGB.bmp,183,75,6
2,0.596065,0.622891,0.642344,0.658734,0.564028,0.547535,0.533123,0.684341,0.516567,0.638584,...,0.769346,0.773790,0.554134,0.878325,0.723066,2.0,102_26_FRAME_85_RGB.bmp,120,39,6
3,0.678144,0.685466,0.684164,0.682298,0.690156,0.690966,0.692118,0.678675,0.694333,0.669507,...,0.772577,0.658132,0.651741,0.790947,0.802804,3.0,104_53_FRAME_103_RGB.bmp,156,56,2
4,0.776119,0.762512,0.752715,0.742342,0.784705,0.790155,0.795267,0.718975,0.793387,0.757359,...,0.744421,0.839728,0.683384,0.418380,0.296281,4.0,104_53_FRAME_70_RGB.bmp,237,130,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,0.526797,0.553086,0.570027,0.586724,0.519070,0.507619,0.497154,0.624617,0.510767,0.553512,...,0.913630,0.671902,0.803484,0.920780,0.908565,486.0,98_845_FRAME_99_RGB.bmp,126,55,6
487,0.460099,0.483845,0.503176,0.522873,0.431644,0.412627,0.393493,0.554600,0.375764,0.502140,...,0.015947,0.026906,0.019161,0.021013,0.021215,487.0,9_859_FRAME_148_RGB.bmp,144,51,7
488,0.470837,0.488221,0.503243,0.515891,0.429539,0.409857,0.394204,0.537908,0.365208,0.499363,...,0.985711,0.999956,0.989557,0.999997,0.995642,488.0,9_888_FRAME_22_RGB.bmp,264,128,8
489,0.432737,0.454818,0.468800,0.482923,0.426068,0.418026,0.410125,0.533544,0.428180,0.461971,...,0.726305,0.004765,0.157191,0.007496,0.587474,489.0,9_888_FRAME_27_RGB.bmp,307,146,7


In [49]:
# save landmarks to csv
df_final.to_csv(os.path.join("../data", "results.csv"), index=False)